# 🧠 Dasein Examples: Universal Memory for Agentic AI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nickswami/dasein-core/blob/main/examples/dasein_examples.ipynb)

This notebook demonstrates **Dasein** - a universal memory system that learns from your agent's execution history and automatically improves performance, reduces costs, and increases reliability.

## What You'll Learn

1. **SQL Agent** - Learn query patterns for database interactions
2. **Browser Agent** - Learn web scraping and navigation strategies
3. **Deep Research Agent** - Optimize multi-agent research workflows

## Key Features

- ✨ **Zero-friction integration** - Wrap any agent in one line
- 🧠 **Automatic learning** - Agents learn from successes and failures
- 📊 **Performance tracking** - See token usage, timing, and improvement metrics
- 🔄 **Retry logic** - Intelligent retry with learned optimizations
- ☁️ **Cloud-powered** - Distributed rule synthesis and storage


## Installation


In [ ]:
# Install Dasein and dependencies
!pip install -q dasein-core

# For browser agent example (Use Case 2)
!pip install -q playwright
!playwright install chromium

# For deep research agent (Use Case 3)
!pip install -q git+https://github.com/langchain-ai/open_deep_research.git


## Setup API Keys

Replace the placeholders below with your actual API keys:

- **GOOGLE_API_KEY**: Get from [Google AI Studio](https://makersuite.google.com/app/apikey)
- **TAVILY_API_KEY**: Get from [Tavily](https://tavily.com) (free tier available, 1000 searches/month)


In [ ]:
import os

# ⚠️ REPLACE THESE WITH YOUR ACTUAL API KEYS
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE"
os.environ["TAVILY_API_KEY"] = "YOUR_TAVILY_API_KEY_HERE"  # Only needed for Use Case 3

# Suppress warnings
os.environ["GOOGLE_CLOUD_DISABLE_DIRECT_PATH"] = "true"

print("✅ API keys configured")
print("\n⚠️ Note: You may see dependency warnings in Colab - these are safe to ignore.")
print("⚠️ Note: Brief timeout warnings on first run are normal (cold start) - services will activate automatically.")


---

# Use Case 1: SQL Agent with Learning

This example demonstrates how Dasein learns SQL query patterns and improves database agent performance.

**What Happens:**
1. **Baseline Run**: Agent queries the Chinook database without optimization
2. **Learning**: Dasein captures the execution trace and sends it to cloud services for rule synthesis
3. **Enhanced Run**: Agent runs again with learned rules injected, showing improved performance

**Expected Improvements:**
- 30-50% reduction in tokens used
- Fewer SQL query iterations
- Better query formulation


In [ ]:
import urllib.request
from pathlib import Path

# Download Chinook sample database
db_path = Path("chinook.db")
if not db_path.exists():
    print("Downloading Chinook database...")
    urllib.request.urlretrieve(
        "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite",
        str(db_path)
    )
    print("✅ Database downloaded")
else:
    print("✅ Database already exists")


In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from dasein import cognate

# Set up SQL agent
print("Setting up SQL agent...")
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True, agent_executor_kwargs={'handle_parsing_errors': True})

print("✅ SQL agent created")


In [ ]:
# Wrap with Dasein for automatic learning and improvement
agent = cognate(
    agent,
    retry=2,  # Run twice: baseline + enhanced
    performance_tracking=True,  # Show improvement metrics
    verbose=False  # Set to True to see rule injection details
)

# Run the query - Dasein handles everything automatically!
query = "Which media format accounts for the highest number of individual line-item sales?"

print(f"\nQuery: {query}\n")
print("=" * 70)
print("PHASE 1: Baseline run...")
print("PHASE 2: Enhanced run with learned rules...")
print("=" * 70)

result = agent.invoke({"input": query})

print(f"\n✅ Final Answer: {result.get('output', '')}")
print("\n📊 Check the performance metrics above to see improvement!")


---

# Use Case 2: Browser Agent with Learning

This example demonstrates how Dasein learns web scraping and navigation patterns.

**What Happens:**
1. **Baseline Run**: Browser agent navigates to a website without optimization
2. **Learning**: Dasein learns which tools to use, navigation patterns, and element selection strategies
3. **Enhanced Run**: Agent runs again with learned rules, showing faster and more efficient browsing

**Expected Improvements:**
- Fewer navigation steps
- Better tool selection
- More efficient element targeting
- Reduced token usage


In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from dasein import cognate
import asyncio
from playwright.async_api import async_playwright

async def create_browser_agent():
    """Create a Playwright browser agent."""
    print("Setting up Playwright browser agent...")
    
    # Create browser
    playwright = await async_playwright().start()
    async_browser = await playwright.chromium.launch(headless=True)
    
    # Create toolkit and tools
    toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
    tools = toolkit.get_tools()
    
    print(f"Available browser tools: {[tool.name for tool in tools]}")
    
    # Create LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        request_timeout=180,
        max_retries=2
    )
    
    # Create agent
    agent_chain = create_react_agent(model=llm, tools=tools)
    agent_chain = agent_chain.with_config({"recursion_limit": 75})
    agent_chain.llm = llm  # Add LLM reference for Dasein
    
    print("✅ Browser agent created")
    return agent_chain

# Create the agent
browser_agent = await create_browser_agent()


In [ ]:
# Wrap with Dasein
browser_agent = cognate(
    browser_agent,
    retry=2,
    performance_tracking=True,
    verbose=False  # Set to True to see rule injection details
)

# Test query
query = "Go to the LangChain site (https://python.langchain.com). Is there a tutorial on how to create a browsing agent? If so where is it?"

print(f"\nQuery: {query}\n")
print("=" * 70)
print("PHASE 1: Baseline run...")
print("PHASE 2: Enhanced run with learned rules...")
print("=" * 70)

result = await browser_agent.ainvoke({"messages": [("user", query)]})

print(f"\n✅ Final Answer: {result}")
print("\n📊 Check the performance metrics above to see improvement!")


---

# Use Case 3: Deep Research Agent with Learning

This example demonstrates Dasein with LangChain's **Open Deep Research Agent** - a complex multi-agent system that orchestrates research tasks.

**What Happens:**
1. **Baseline Run**: Multi-agent research system runs without optimization
   - Supervisor delegates to sub-agents
   - Parallel research with Tavily search
   - Context accumulation and synthesis
   - Typically uses 15x more tokens than normal chat!

2. **Learning**: Dasein learns:
   - Better search query formulation
   - Context filtering to reduce token bloat
   - Improved delegation strategies
   - Efficient tool usage patterns

3. **Enhanced Run**: Agent runs with learned rules

**Expected Improvements:**
- 20-40% reduction in tokens used
- Fewer redundant searches
- Better query formulation
- Faster convergence to quality results

**Requirements:**
- TAVILY_API_KEY (free tier: 1000 searches/month)


In [ ]:
# Critical fix: Force LangChain to use Google GenAI instead of Vertex AI
from langchain.chat_models import base as chat_models_base
from langchain_google_genai import ChatGoogleGenerativeAI

original_init_helper = chat_models_base._init_chat_model_helper
_model_cache = {}
_patch_logged = set()

def patched_init_helper(*args, **kwargs):
    """Force gemini models to use ChatGoogleGenerativeAI."""
    model = kwargs.get('model', '')
    if not model and args:
        model = args[0] if len(args) > 0 and isinstance(args[0], str) else ''
    
    if model and 'gemini' in model.lower():
        if model not in _patch_logged:
            print(f"🔧 Using ChatGoogleGenerativeAI for {model}")
            _patch_logged.add(model)
        
        if model not in _model_cache:
            _model_cache[model] = ChatGoogleGenerativeAI(model=model)
        return _model_cache[model]
    return original_init_helper(*args, **kwargs)

chat_models_base._init_chat_model_helper = patched_init_helper
print("✅ Patched LangChain to use Google GenAI")


In [ ]:
from open_deep_research.deep_researcher import deep_researcher_builder
from open_deep_research.configuration import Configuration
from dasein import cognate
import asyncio

async def create_research_agent():
    """Create Open Deep Research agent."""
    print("Setting up Open Deep Research Agent...")
    print("Source: https://github.com/langchain-ai/open_deep_research\n")
    
    # Configure with Gemini models and Tavily search
    config = Configuration(
        summarization_model="gemini-2.5-flash",
        research_model="gemini-2.5-flash",
        compression_model="gemini-2.5-flash",
        final_report_model="gemini-2.5-flash",
        allow_clarification=False,
        max_concurrent_research_units=2,
        max_researcher_iterations=4,
        max_react_tool_calls=8,
    )
    
    print("Configuration:")
    print(f"  - Models: {config.research_model}")
    print(f"  - Search API: Tavily")
    print(f"  - Max concurrent researchers: {config.max_concurrent_research_units}\n")
    
    # Create the deep researcher graph
    graph = deep_researcher_builder.compile()
    
    print("✅ Research agent created\n")
    print("Graph structure:")
    print("  1. Brief Generation")
    print("  2. Research Supervisor")
    print("  3. Research Sub-Agents (parallel)")
    print("  4. Report Writing\n")
    
    # Add LLM reference for Dasein
    graph.llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    
    return graph

# Create the research agent
research_agent = await create_research_agent()


In [ ]:
# Wrap with Dasein
research_agent = cognate(
    research_agent,
    retry=2,
    performance_tracking=True,
    verbose=False  # Set to True to see rule injection details
)

# Research query
query = """Research and compare LangChain vs LlamaIndex for building LLM applications.

I want to understand:
1. Their core philosophies and approaches
2. Key features and strengths
3. Which use cases each is best suited for

Provide a concise comparison with sources."""

print(f"Research Query: {query}\n")
print("=" * 70)
print("This will trigger:")
print("  - Supervisor creating 2 sub-agents (one per framework)")
print("  - Parallel research with Tavily search")
print("  - Multiple search rounds per sub-agent")
print("  - Synthesis and report writing\n")
print("PHASE 1: Baseline run...")
print("PHASE 2: Enhanced run with learned rules...")
print("=" * 70)

result = await research_agent.ainvoke(
    {"messages": [("user", query)]},
    config={
        "configurable": {
            "allow_clarification": False,
            "summarization_model": "gemini-2.5-flash",
            "research_model": "gemini-2.5-flash",
        }
    }
)

# Display report (truncated for readability)
report = str(result.get('report', result))
if len(report) > 500:
    print(f"\n✅ Research Report (truncated):\n{report[:500]}...\n[{len(report)} total chars]")
else:
    print(f"\n✅ Research Report:\n{report}")

print("\n📊 Check the performance metrics above to see massive token savings!")


---

## Summary

You've just seen **Dasein** in action across three different agent types:

1. **SQL Agent** - Learned query optimization patterns
2. **Browser Agent** - Learned navigation and tool usage strategies
3. **Deep Research Agent** - Learned to optimize complex multi-agent workflows

### Key Takeaways

- ✨ **One-line integration**: `cognate(agent)` is all you need
- 🧠 **Automatic learning**: Agents improve themselves without manual rule engineering
- 📊 **Transparent metrics**: See exactly how much you're improving
- ☁️ **Cloud-powered**: Rules are stored and synthesized using distributed services
- 🔄 **Intelligent retry**: `retry=2` automatically runs baseline + enhanced

### Next Steps

1. **Try your own agents** - Dasein works with any LangChain/LangGraph agent
2. **Customize optimization** - Use the `weights` parameter to optimize for what matters to you
3. **Build rule libraries** - Run multiple times to accumulate learned rules
4. **Scale up** - The more you use Dasein, the smarter your agents become

### Resources

- 📖 [Documentation](https://github.com/nickswami/dasein-core#readme)
- 💻 [GitHub](https://github.com/nickswami/dasein-core)
- 💬 [Discord Community](https://discord.gg/dasein)
- 🐛 [Report Issues](https://github.com/nickswami/dasein-core/issues)

---

**Built with ❤️ for the agentic AI community**
